> 1. DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.
> 2. SHARE SETTINGS: In the new notebook, set the sharing settings to "Anyone with the link" by clicking "Share" on the top right corner.





Welcome to this second week's project for *Intermediate SQL*!

This week's lecture and material on CoRise showed you how to write joins in SQL and combine that with other things you learned in Week 1. For this project we further your understanding of these concepts by delving a bit deeper. However, for everything covered in this project, you can find related examples in our course material.


# **Prerequisite configuration**
Below we install the software required to run this project. Please make sure to **RUN IT** by clicking on the play-button icon and to feel free to ignore the content of these two hidden cells.

In [ ]:
%%capture
!pip install git+https://github.com/corise-edu/course-intermediate-sql.git

In [ ]:
import pandas as pd
from IPython.display import display, HTML
from sql_course import run as sql_run
from sql_course import check

# Show all the rows (instead of only a few)
pd.set_option("display.max_rows", None)

# Set precision to max 2 decimals
pd.set_option('precision', 2)

# Set CSS Style for Table
# Make it work with night & light mode
# - Alternating rows
# - th elements
# - td elements
css_style = '''
<style>
  html {
    --td-font-color: black;
    --font-color: black;
    --background-color: #e0e0e0;
  }
  html[theme=dark] {
    --td-font-color: white;
    --font-color: black;
    --background-color: #6688ff;
  }
  th {
    background: #fbd44c;
    color: var(--font-color);
    font-size: 16px;
    text-align: center;
    font-weight: bold;
  }
  tr:nth-child(even) {
    background-color: var(--background-color);
    color: var(--font-color);
  }
  td {
    font-size: 14px;
    color: var(--td-font-color);
  }
</style>
'''


def run(sql_query):
  df = sql_run(sql_query)

  # Puts the scrollbar next to the DataFrame
  display(HTML(css_style +
               "<div style='max-height: 500px; overflow: auto; width: fit-content; border-style: solid;" +
               " border-width: 1px; border-color: #0139fe; font-family: GT Planar,Inter,Arial,sans-serif;'>" +
               df.style.render() +
               "</div>"))

# **The Setting:**

Your career at CoRise has continued to grow. As the complexity has increased in their database tracking course-related information, your role has expanded to include proactively looking for patterns in the data.

You are storing information in an interrelated set of tables now, and you need to bring out your ace SQL skills to satisfy your data curiosity.

We will be going through a series of analyses to identify avenues for growing the impact of the awesome courses that are coming out of CoRise. As before, we are focused on nps as the key metric to evaluate how our courses are being received by the learners. We want to continue to explore the possible correlation of various data elements with the nps.


# **Schema:**

As we discussed in the lessons, the `courses` table from Week 1 has been replaced with the five tables below:


```
create table course_info (
course_id integer,
course_name text,
course_desc text,
course_category text,
course_level text
);
```

```
create table instructors (
instructor_id integer,
name text,
affiliation text,
teaching_experience integer
);
```

```
create table learners (
learner_id integer,
name text,
affiliation text
);
```

```
create table course_run (
course_run_id integer,
course_id integer,
instructor_id integer,
start_date date,
num_weeks integer,
num_TAs integer,
nps integer
);
```

```
create table course_registration_info (
course_run_id integer,
learner_id integer
);
```

# **Disclaimer**

Any resemblance of the data in this table to anything real (other than the names and descriptions of courses) is entirely random (could have been my pseudo random generator or my own imagination, since it isn't easy to distinguish between the two :-), so do not stress yourself out trying to make sense of any categorization, such as why a course run might even overlap with another run and other such niceties).

# **Preamble**

Let's begin by revisiting some queries that we did on the `courses` table last week, except this week you will likely have to combine multiple tables to answer those queries! The first three parts of this week's project are identical to the corresponding sections from last week. This will hopefully serve to contrast how the same English query translates to SQL depending on the schema (how normalized it is or isn't).

Please take care to not use the `courses` table from last week in any of your queries. You don't need that table any longer - in fact the goal of this week's project is to work using the normalized schema described above. Although you may be able to get correct answers for some of the queries by using the old `courses` table, please ensure that you're using the new table names this week.

# **Part 1 - Digging into Machine Learning Courses:**



You have heard from your boss that Machine Learning is a very important category for the CoRise business, and it would be very good to keep expanding the scope of the courses in this broad area. But first, we should understand reality as it is today. So, your logical next question:




In [ ]:
### Question: q2_1_1
### For the course category 'Machine Learning,'
### how many courses have we run so far?
### Output columns: cnt
query = """
    SELECT count(*) as cnt FROM course_info join course_run
    on (course_info.course_id = course_run.course_id)
    WHERE course_category= 'Machine Learning';
         """

run(query)
check(q2_1_1 = query)



,cnt
0,310


-------------------
Your SQL query is correct!


It's useful information to be able to see that we have run 310 courses so far. To dig deeper, you decide to look at how many courses we have run for each year to get an idea of that distribution.

In [ ]:
### Question: q2_1_2
### For the course category 'Machine Learning,'
### how many courses have we run so far for each calendar year?
### Output Columns: year, count_by_year

query = """
           SELECT strftime("%Y", start_date) as year, count(course_category) as count_by_year FROM course_run join course_info
    on (course_run.course_id = course_info.course_id)
    WHERE course_category= 'Machine Learning'
    GROUP BY year;

         """

run(query)
check(q2_1_2 = query)

,year,count_by_year
0,2016,35
1,2017,45
2,2018,37
3,2019,39
4,2020,51
5,2021,38
6,2022,33
7,2023,32


-------------------
Your SQL query is correct!


Hmm... Anecdotally you seem to recollect that we are running 60+ courses a year that can be termed 'Machine Learning.' The general talk around the water cooler is that we seem to have 5 ML courses a month. We would need to be running quite a few more courses to get to 60 per year. One thing that comes to your notice is that there are perhaps courses that are not filed under the 'Machine Learning' category but could still be considered 'Machine Learning.' You decide to look for some words in the name and description to potentially infer other courses that you could consider 'Machine Learning.'

In [ ]:
### Question: q2_1_3
### Retrieve distinct combinations of name, category and descriptions of courses
### that contain one of the words 'ml' or 'learning' or 'models'
### We want the match to be case-insensitive.
### Note that in general this need not be the same set of courses
### as ones that belong to the category 'Machine Learning'.
### Output Columns: course_name, course_category, course_desc

query = """
                SELECT DISTINCT course_name, course_category, course_desc  FROM course_info join course_run
                    on (course_info.course_id = course_run.course_id)
      WHERE lower(course_name) LIKE '%ML%'OR  lower(course_name)
       LIKE '%models%' or lower(course_name) LIKE '%learning%' or lower(course_category) LIKE '%Machine Learning%' OR lower(course_desc) LIKE '%ML%' or lower(course_desc) LIKE '%models%' or lower(course_desc) LIKE '%learning%';


         """

run(query)
check(q2_1_3 = query)

,course_name,course_category,course_desc
0,Spoken Language Processing,Language Processing,Learn to design and build voice assistant or voice cloning systems;using available APIs and adjustable pre-trained models;as well as creating them from scratch in PyTorch. You will learn solid audio processing fundamentals combined with the most important advances in automatic speech recognition and speech synthesis methods.
1,Social Media Mining,Language Processing,Collect;analyze;and present insights from social media data — including the Twitter API — while learning concepts of natural language processing and graph analytics.
2,How to Implement ML Papers,Machine Learning,Learn how to implement the proposed algorithms;models;and techniques from ML papers each week;while learning tips and tricks on debugging the implementation efforts and how to reproduce results in your own applications.
3,Python for Data Science,Machine Learning,Build projects like computing your own heart rate from an ECG;building a Shazam music identifier clone;and more while learning how to use core python libraries like Numpy;Scipy;Scikit;and Matplotlib.
4,MLOps: From Models to Production,Machine Learning,Acquire the skills to build effective real-world ML systems (bootstrapping datasets;improving label quality;experimentation;model evaluation;deployment and observability) with hands-on projects. This course will help you bridge the gap between state-of-the-art ML modeling;and building real-world ML systems.
5,Applied Machine Learning,Machine Learning,Design;build;and debug machine learning models for classification and regression tasks using a variety of datasets with Python (Numpy;Scikit;Pyplot). Learn best practices to plan and execute ML development projects whether large or small.
6,Search with Machine Learning,Machine Learning,We have designed this course to cover the fundamentals of integrating machine learning and natural language processing techniques into search engines. We will dive into using machine learning for ranking;content understanding;and query understanding;along with how to use embeddings;dense vectors and deep learning to improve retrieval and ranking.
7,Natural Language Processing,Language Processing,Build your own auto-compose tool and other projects while learning key NLP concepts such as BERT;entity recognition;intent classification;spacy;and Transformer models powering search engines like Google or voice assistants like Google Home or Amazon Alexa.
8,Fundamentals of Data Modeling,Data Engineering,This course provides an introduction to the fundamentals of data modeling for modern data warehouses. We use the Kimball dimensional model to guide us;but that does not mean this class is all theory! We have designed this course so you will learn the theory behind data modeling (normalization / denormalization;star schemas;fact and dimension tables) and you will build actual models using real world data. We will use plain SQL so you can do modeling whether you only have access to SQL or you are using a tool like dbt. By the end of this course;you will have a deep understanding of dimensional modeling;and you will have built one from scratch.
9,Building Computer Vision Applications,Machine Learning,This course provides an introduction to machine learning for computer vision with a focus on practical applications relevant to industry teams. In this course;we will “reverse-engineer” a number of applications;such as traffic flow analysis;digital medicine;optical character recognition;and video analytics.


-------------------
Your SQL query is correct!


You found a few more courses that are in the 'Deep Learning' category and in the 'Language Processing' category that could be considered 'Machine Learning,' but you also found other courses that refer to data models under the 'Data Engineering' category. You decide to cross-check what would happen if you were to also count the courses run per year in the categories 'Deep Learning' and 'Language Processing.'

In [ ]:
### Question: q2_1_4
### For the course category 'Machine Learning', 'Deep Learning' & 'Language Processing'
### how many courses have we run so far for each calendar year?
### Output columns: year, count_by_year

query = """
           SELECT strftime("%Y", start_date) as year, count(course_category) as count_by_year FROM course_run join course_info
    on (course_run.course_id = course_info.course_id)
    WHERE course_category= 'Machine Learning' or  course_category='Deep Learning' or course_category= 'Language Processing'
    GROUP BY year;
        """

run(query)
check(q2_1_4 = query)

,year,count_by_year
0,2016,60
1,2017,73
2,2018,61
3,2019,76
4,2020,80
5,2021,65
6,2022,57
7,2023,63


-------------------
Your SQL query is correct!


That seems to align better with the anecdotal water cooler conversations. The head of content is happy that you have been able to paint a good picture of current reality and is now off trying to figure out what new courses to add and what instructors they need to engage. But your life as an analyst continues. The CEO has put together a special ops commando team called the 'COVID' impact team. Of course, you are part of this team as the ace analyst in CoRise. Off we go!

# **Part 2 - COVID Impact**

As mentioned before, the CEO has put together a special ops commando team called the 'COVID' impact team which includes you as the star analyst. No one is sure if the courses were impacted by COVID as it has been a while. (You are in 2024 and the world has almost forgotten COVID (positive dreams are good for us 😀)). You suggest that you will mine the data and see if there are any quantitative differences between 2020 and other years.  

As you know by now, we are guided by nps quite a bit, as it is a great measure of how the course was perceived by the participants. So the first thing is to see if there is any substantial difference either in the total quantity of the courses or the average nps of the courses.


In [ ]:
### Question: q2_2_1
### Print the number of courses run per year.
### Output columns: year, count_by_year
query = """
           SELECT strftime("%Y", start_date) as year, count(course_name) as count_by_year FROM course_info join course_run
    on (course_info.course_id = course_run.course_id)
    GROUP BY year;
        """

run(query)
check(q2_2_1 = query)

,year,count_by_year
0,2016,129
1,2017,118
2,2018,128
3,2019,134
4,2020,140
5,2021,127
6,2022,108
7,2023,115


-------------------
Your SQL query is correct!


That's interesting. The number of courses conducted doesn't seem to have been much lower in 2020. If anything, it is on the higher side. So somehow COVID did not impact the number of courses. Let's look at the average nps next and contrast 2020 with the other years.

In [ ]:
### Question: q2_2_2
### Print a report with two columns titled covid_nps and rest_nps.
### 2020_nps will have the average of the nps of courses that started in 2020 and
### rest_nps will have the average of the nps of courses that started in other years.
### Output columns: covid_nps, rest_nps
query = """
      select
avg(case when strftime("%Y",start_date) = '2020' then nps END) as  covid_nps,
avg(case when strftime("%Y",start_date) != '2020' then nps END) as rest_nps
from  course_info join course_run
on (course_info.course_id = course_run.course_id)
        """

run(query)
check(q2_2_2 = query)

,covid_nps,rest_nps
0,65.89,75.75


-------------------
Your SQL query is correct!


Wow! Thats's a clear drop in the average nps in 2020 compared to the average nps of other years. The statistician in you wonders if there are other years where the nps is low and is compensated for some other years where nps is high - thus 2020 may not be an exception. Let's quickly cross-validate that thought!

In [ ]:
### Question: q2_2_3
### For each year, print the year and the average nps of the courses that started that year.
### Output columns: year, avg(nps)

query = """
select   strftime("%Y",start_date) as year, avg(nps)
from  course_info join course_run
on (course_info.course_id = course_run.course_id)
GROUP BY year
        """

run(query)
check(q2_2_3 = query)

,year,avg(nps)
0,2016,75.76
1,2017,77.58
2,2018,75.56
3,2019,75.93
4,2020,65.89
5,2021,73.93
6,2022,75.99
7,2023,75.67


-------------------
Your SQL query is correct!


Ok, good! 2020 was an aberration. The logical next question is why? Are there any explanatory variables? You go talk to some folks who have been around in the company for a long time. They tell you they recollect that it was tough getting enough TAs for many courses in 2020. Additionally, the courses ran for longer to account for sickness of many of the staff as well as learners. You set out to validate this next.

Argh... It turns out that expressing the query from last time (q_3_4 from week 1)  using our current SQL knowledge is not so straightforward. While it can be done, it turns out it's very convoluted. We will skip this for now.

# **Part 3 - Looking for Pockets of Underperformance:**

The head of content creation would like to understand the distribution of nps overall to get their bearings (recall that this is the most important metric for the business).

In [ ]:
### Question: q2_3_1
### Find the minimum, maximum, and average  of nps across all courses
### Output columns: min_nps, max_nps, avg_nps
query = """
select  max(nps) as max_nps, min(nps) as min_nps, avg(nps) as avg_nps
from  course_info join course_run
on (course_info.course_id = course_run.course_id)
         """

run(query)
check(q2_3_1 = query)

,max_nps,min_nps,avg_nps
0,100,40,74.37


-------------------
Your SQL query is correct!


At first blush, the news is good – the average seems to be reasonably high. Of course it would always be good to be better, but it's a start. However, the minimum is quite low. Perhaps there is a pocket of courses that can be improved and we can lift the average that way. You decide to dig deeper and see if you can find pockets of underperformance. You instintctively want  to explore if the size of a course (number of learners) has some correlation with the nps - this is q_4_2 from week 1. Unfortunately this query is also complicated to do with the set of things we have learned so far. We will revisit this next week with subqueries.

Meanwhile, you have been joining the head of course content in some feedback interviews with learners. The positive interviews always seems to have two flavors: a few learners preferred courses that ran longer and dug deeper into an area, while others preferred courses that were quick and exposed them to a new area. You decide to investigate this and see if somehow the nps was correlated to this combination of the length of the course and the level of the course.

In [ ]:
### Question:q2_3_2
### Let's designate courses that are fewer than 4 weeks to be 'short'
### and all the others 'long' and call this course_duration.
### Let's get the average nps for each course_level and course_duration combination.
### Note there should be four rows in your output given that
### we have two values for course_duration and two values for course_level.
### Output columns: course_duration, course_level, avg_nps
query = """
SELECT CASE WHEN num_weeks <4 then 'short' else 'long' END as course_duration, avg(nps) as avg_nps, course_level
from  course_info join course_run
on (course_info.course_id = course_run.course_id)
GROUP BY course_duration, course_level
      """

run(query)
check(q2_3_2 = query)

,course_duration,avg_nps,course_level
0,long,77.95,A
1,long,69.91,B
2,short,69.44,A
3,short,80.74,B


-------------------
Your SQL query is correct!


Aha! Indeed the advanced courses seem to make more sense when they are run longer. Perhaps the learners feel they are able to dig into the subject. Conversely, shorter basic courses are more valuable - just a quickstart or a crash course to get introduced to a new area so they can come back to advanced courses after digesting the basic course quickly. A great analysis by you and something that has made a huge dent in understanding the course data!

# **Part 4 - Bonus Questions: Troubleshooting and Analytics for Enterprise Customers**

This section is bonus and the queries are a bit more complex in this section.

One of your main jobs as an analyst is to handle one-off requests for data and analysis from your customer-facing sales teams. CoRise works with a number of enterprise customers who send groups of learners as well as supply top quality instructors to CoRise. As a result, it's important to equip the CoRise sales teams with the right kind of information and insights.

The account manager for M & T Bank Corporation wants to understand the nps of basic courses attended by more than just a few learners from M & T Bank Corporation provided the instructor is not from M & T Corporation.





In [ ]:
### Question:q2_4_1
### Get the course_name, course_run_id, number of learners from M & T Bank, & nps
### for each course run that has more than 2 learners from M & T Bank
### and is taught by an instructor who isn't from M & T Bank Corporation
### for a basic course (course level = 'B').
### Hint: This requires you to join all five tables from this week's project schema.
### Hint: To get the number of learners you will need to group by all other columns
### in the select clause and use count to get to the number of learners.
### Output Columns: course_name, CR.course_run_id, num_learners, nps
query = """


"""

run(query)
check(q2_4_1 = query)



DatabaseError: ignored

There seems to have been only one course where the nps was particularly low. The account manager sets off to learn more from the course manager about what may have happened during that course. Then they will be able to communicate to their customer that the courses for their learners are predominantly going well, and they may also be able to educate them on the reasons this course was worse and collect feedback from the customer on that course as well.

After a few months, the account manager from Cincinnati Financial Corp. is wondering how courses run by instructors from Cincinnati Financial Corp. are performing. In particular, they want to know which course runs with an instructor from Cincinnati Financial Corp. have been followed by another run of the same course (taught by anyone) later but still within the same calendar year or the next calendar year with a substantially higher nps (difference of at least 10).

In [ ]:
### Question:q2_4_2
### Which course runs (CR1.course_run_id) with an instructor from Cincinnati
### Financial Corp. have been followed by another run (CR2.course_run_id,
### any instructor affiliation) of the same course (same course_id)
### later but still within the samevcalendar year or the next calendar year
### with a substantially higher nps (difference of at least 10).
### Hint: Recall that we did a self join here in this module (url below).
### It may be useful to review that info here:
### https://corise.com/course/intermediate-sql/v2/module/week-2-joins-involving-more-than-two-tables-3hx8j#corise_cl953jnuf00083b73cuijvbhw
### Output Columns: CR1.course_run_id, CR2.course_run_id, CR1.nps, CR2.nps,
###                 I1.affiliation, I2.affiliation, CR1.start_date, CR2.start_date
query = """



"""

run(query)
check(q2_4_2 = query)

Your account managers cannot thank you enough for the ammunition you have provided for their conversations with their customers. This has been hugely helpful.

# **Conclusion:**

Wow! You have become a master at combining tables. This means you are ready to drive serious impact on any set of tables in a production environment.